In [2]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
import pandas as pd
from pprint import pprint
import openweathermapy.core as owm
import seaborn as sns

In [4]:
import numpy as np

np_lat = np.arange(-90,90,10)
np_lng = np.arange(-180,180,18)
# coordinates = (np_lat,np_lng) #Better keeping lat/lng in separate columns

import random

rnd_lat = random.choice(np_lat)
rnd_lng = random.choice(np_lng)
# rnd_coordinates = random.choice(coordinates) #Better keeping lat/lng in separate columns

from citipy import citipy

weather_df = pd.DataFrame()
weather_df['rnd_lat']=""
weather_df['rnd_lng']=""

for x in np_lat:
    for y in np_lng:
        x_values = list(np.arange(-90,90,0.01))
        y_values = list(np.arange(-180,180,0.01))
        lats = random.sample(x_values,17999) #why 17999 instead of 500? Because of duplicates..
        lons = random.sample(y_values,17999) #..stemming from the geocoords from citipy
                                            #and that's the maximum number of samples I can input
                                            #given the latitude parameters (180/0.01 = 18000)
                                            #Also I believe a larger sampling size to be better to
                                            #increase the random distribution of values.
        
weather_df = weather_df.append(pd.DataFrame.from_dict({"rnd_lat":lats,
                                       "rnd_lng":lons}))

weather_df = weather_df.reset_index(drop=True)



weather_df['city']=""
weather_df['country_code']=""

for index,row in weather_df.iterrows():
        weather_df_city = citipy.nearest_city(row["rnd_lat"], row["rnd_lng"])
        weather_df.set_value(index,"city",weather_df_city.city_name)
        weather_df.set_value(index,"country_code",weather_df_city.country_code)
        
weather_df=weather_df.drop(['rnd_lat', 'rnd_lng'],axis=1) #geo coord don't correspond to cities

weather_df=weather_df.drop_duplicates() #hmmm, some coords might yield similar cities....
print("Sampling size (from n = 17,999) after dumping duplicates: n=" + str(len(weather_df)))

weather_df = weather_df.sample(800) #resample so that n=800 (why? because some cities will may lack
                                    # data for certain attributes, so we'll re-sample once data has
                                    # dropped any rows with NULL values for any attributes)
weather_df = weather_df.reset_index(drop=True) #Remove index pre-resampling

print(weather_df.head())

print("Confirm n = 800: " + str(weather_df.shape))

C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Sampling size (from n = 17,999) after dumping duplicates: n=3362
         city country_code
0      darhan           mn
1        kyra           ru
2   paraopeba           br
3    souillac           mu
4  slave lake           ca
Confirm n = 800: (800, 2)


In [19]:
from config2 import api_key 

    # Set parameters to pass in query URL
units = "metric" 
params = {
    "appid": api_key,
    "units": "metric",
}

    # Set counter for request iteration for each city
counter = 0

    # Set Request Loop
for index, row in weather_df.iterrows():
    params["q"] =f'{row["city"]},{row["country_code"]}' #Set location params: city, country_code
    target_url = 'http://api.openweathermap.org/data/2.5/weather?' #i.e. OWM's base API url

    counter += 1 # add to the index count for each iterations
#     print("Now retrieving city # " + str(counter)) #making sure the counter iteration is working
    
    city_owm = requests.get(target_url, params=params)
    print("ID:"+ str(counter) + " " + "OWM info for " + str( params["q"]))
    print(city_owm.url) #making sure the request iterations are working for each location.
    
    city_owm  = city_owm.json()
    
    #Pull attributes of interest from OWM request
    weather_df.set_value(index,"Latitude",city_owm.get("coord",{}).get("lat"))
    weather_df.set_value(index,"Longitude",city_owm.get("coord",{}).get("lon"))
    weather_df.set_value(index,"Temperature",city_owm.get("main",{}).get("temp_max"))
    weather_df.set_value(index,"Wind_speed",city_owm.get("wind",{}).get("speed"))
    weather_df.set_value(index,"Humidity",city_owm.get("main",{}).get("humidity"))
    weather_df.set_value(index,"Cloudiness",city_owm.get("clouds",{}).get("all"))

ID:1 OWM info for darhan,mn
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=darhan%2Cmn


C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\

ID:2 OWM info for kyra,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=kyra%2Cru
ID:3 OWM info for paraopeba,br
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=paraopeba%2Cbr
ID:4 OWM info for souillac,mu
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=souillac%2Cmu
ID:5 OWM info for slave lake,ca
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=slave+lake%2Cca
ID:6 OWM info for meulaboh,id
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=meulaboh%2Cid
ID:7 OWM info for oranzherei,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=oranzherei%2Cru
ID:8 OWM info for zakamensk,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metri

ID:58 OWM info for barinitas,ve
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=barinitas%2Cve
ID:59 OWM info for vangaindrano,mg
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=vangaindrano%2Cmg
ID:60 OWM info for lahaina,us
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=lahaina%2Cus
ID:61 OWM info for borogontsy,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=borogontsy%2Cru
ID:62 OWM info for saint-gaudens,fr
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=saint-gaudens%2Cfr
ID:63 OWM info for maniitsoq,gl
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=maniitsoq%2Cgl
ID:64 OWM info for teya,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c

ID:115 OWM info for merrill,us
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=merrill%2Cus
ID:116 OWM info for puerto asis,co
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=puerto+asis%2Cco
ID:117 OWM info for daru,pg
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=daru%2Cpg
ID:118 OWM info for conceicao do araguaia,br
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=conceicao+do+araguaia%2Cbr
ID:119 OWM info for tateyama,jp
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=tateyama%2Cjp
ID:120 OWM info for trollhattan,se
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=trollhattan%2Cse
ID:121 OWM info for malwan,in
http://api.openweathermap.org/data/2.5/weather?appid=9f87482

...

ID:563 OWM info for saint-augustin,ca
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=saint-augustin%2Cca
ID:564 OWM info for hongjiang,cn
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=hongjiang%2Ccn
ID:565 OWM info for singarayakonda,in
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=singarayakonda%2Cin
ID:566 OWM info for sodertalje,se
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=sodertalje%2Cse
ID:567 OWM info for mahebourg,mu
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=mahebourg%2Cmu
ID:568 OWM info for venkatagiri,in
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=venkatagiri%2Cin
ID:569 OWM info for savonlinna,fi
http://api.openweathermap.org/data/2.5/weather

ID:619 OWM info for boyabat,tr
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=boyabat%2Ctr
ID:620 OWM info for porangatu,br
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=porangatu%2Cbr
ID:621 OWM info for aloleng,ph
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=aloleng%2Cph
ID:622 OWM info for majene,id
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=majene%2Cid
ID:623 OWM info for san miguel de cauri,pe
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=san+miguel+de+cauri%2Cpe
ID:624 OWM info for issoire,fr
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=issoire%2Cfr
ID:625 OWM info for porto velho,br
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739

ID:675 OWM info for khandyga,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=khandyga%2Cru
ID:676 OWM info for tome-acu,br
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=tome-acu%2Cbr
ID:677 OWM info for son tay,vn
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=son+tay%2Cvn
ID:678 OWM info for sawakin,sd
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=sawakin%2Csd
ID:679 OWM info for kolosovka,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=kolosovka%2Cru
ID:680 OWM info for nara,ml
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=nara%2Cml
ID:681 OWM info for poso,id
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q

ID:731 OWM info for osa,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=osa%2Cru
ID:732 OWM info for mezhdurechensk,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=mezhdurechensk%2Cru
ID:733 OWM info for boda,cf
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=boda%2Ccf
ID:734 OWM info for pinega,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=pinega%2Cru
ID:735 OWM info for chamba,in
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=chamba%2Cin
ID:736 OWM info for lampa,cl
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=lampa%2Ccl
ID:737 OWM info for jacareacanga,br
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=j

ID:788 OWM info for semnan,ir
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=semnan%2Cir
ID:789 OWM info for sampit,id
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=sampit%2Cid
ID:790 OWM info for vanino,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=vanino%2Cru
ID:791 OWM info for lere,ng
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=lere%2Cng
ID:792 OWM info for kaoma,zm
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=kaoma%2Czm
ID:793 OWM info for iskateley,ru
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=iskateley%2Cru
ID:794 OWM info for tabiauea,ki
http://api.openweathermap.org/data/2.5/weather?appid=9f87482612efa739e32c116922d4daac&units=metric&q=tabiauea%

In [5]:
weather_df = weather_df.dropna()
# weather_df = weather_df.sample(500) #resample so that n=500
print(weather_df.shape)
weather_df.head()


# import csv
weather_df.to_csv("owm_sample.csv",index=False)

# Save to df
weather_csv = "owm_sample.csv"
weather_df = pd.read_csv(weather_csv)

In [6]:
weather_csv = "owm_sample.csv"
weather_df = pd.read_csv(weather_csv)
weather_df.to_pickle("weather_df.pkl")
weather_df=pd.read_pickle("weather_df.pkl")
print(weather_df.columns)
print(weather_df.shape)
weather_df.head()

weather_df_lat = weather_df['Latitude']
weather_df_temp = weather_df['Temperature']
weather_df_hum = weather_df.Humidity
weather_df_clo = weather_df.Cloudiness
weather_df_wind = weather_df.Wind_speed

df = pd.DataFrame({'Latitude': weather_df.Latitude,
                   'Temperature (°C)': weather_df.Temperature,
                   'Wind speed (mph)': weather_df.Wind_speed,
                  'Humidity (%)': weather_df.Humidity,
                   'Cloudiness (%)': weather_df.Cloudiness
                  })

df.head()

Index(['city', 'country_code', 'Latitude', 'Longitude', 'Temperature',
       'Wind_speed', 'Humidity', 'Cloudiness'],
      dtype='object')
(716, 8)


,Latitude,Temperature (°C),Wind speed (mph),Humidity (%),Cloudiness (%)
0,49.49,5.23,1.16,86.0,0.0
1,49.58,13.28,0.96,89.0,48.0
2,-19.28,25.00,4.10,47.0,75.0
3,-20.52,23.00,6.70,73.0,75.0
4,55.28,15.43,1.81,72.0,76.0


In [25]:
# Histogram of the distribution of sampled Latitudes
    # Plotly tutorial: https://plot.ly/python/histograms/

print(weather_df_lat.describe())
    
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

data = [go.Histogram(y=weather_df_lat)]
#######
#     Reference: https://plot.ly/python/shapes/

layout = {
    'xaxis': {
        'range': [0, 40]
    },
    'yaxis': {
        'range': [-50, 90]
    },
    'shapes': [

# Line Horizontal
        # Equator
        {
            'type': 'line',
            'x0': 0,
            'y0': 0,
            'x1': 40,
            'y1': 0,
            'line': {
                'color': 'rgb(220, 20, 60)',
                'width': 2
            },
        },
    ]}

fig = {
    'data': data,
    'layout': layout,
}
####
py.iplot(fig, filename='horizontal histogram')

count    716.000000
mean      22.800587
std       29.478586
min      -46.280000
25%       -1.437500
50%       28.295000
75%       48.065000
max       72.790000
Name: Latitude, dtype: float64


In [26]:
# Histogram of the distribution of sampled Temperatures

print(weather_df_temp.describe())

import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

data = [go.Histogram(x=weather_df_temp)]

py.iplot(data, filename='basic histogram')

count    716.000000
mean      20.344930
std        7.519356
min       -0.130000
25%       14.317500
50%       21.040000
75%       26.000000
max       36.980000
Name: Temperature, dtype: float64


In [7]:
# Import packages for Regression Plot

import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
from sklearn import datasets, linear_model

In [8]:
# Temperature (C) vs. Latitude

weather_df_temp=weather_df_temp.as_matrix().reshape(-1, 1)
weather_df_lat=weather_df_lat.as_matrix().reshape(-1, 1)

# Plotly regression tutorial : https://plot.ly/scikit-learn/plot-ols/

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(weather_df_lat, weather_df_temp)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(weather_df_lat) - weather_df_temp) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(weather_df_lat, weather_df_temp))

####

def data_to_plotly(x):
    k = []
    
    for i in range(0, len(x)):
        k.append(x[i][0])
        
    return k
####
p1 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=weather_df_temp, 
                mode= 'markers',
                text= weather_df.city + ", " + weather_df.country_code
#                 marker=dict(color='black')
               )

p2 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=regr.predict(weather_df_lat),
                mode='lines',
                text = "Regression Line",
                line=dict(color='red', width=3)
                )

layout = go.Layout(title='Temperature (°C) over Latitude <br> (as of 08/25/2018) <br> Data Source: <a href="https://openweathermap.org/api">OpenWeatherMap</a>',
                xaxis=dict(title='Latitude',ticks='', showticklabels=False,
                              zeroline=False),
                   yaxis=dict(title='Temperature (°C)', ticks='', showticklabels=False,
                              zeroline=False),
                   showlegend=False, hovermode='closest')

fig = go.Figure(data=[p1, p2], layout=layout)

py.iplot(fig,session="knitr")

C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\site-packages\ipykernel_launcher.py:3: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\site-packages\ipykernel_launcher.py:4: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



Coefficients: 
 [[-0.05392584]]
Mean squared error: 53.94
Variance score: 0.04


In [11]:
# Humidity (%) vs. Latitude

# print(weather_df_hum.describe())

weather_df_hum=weather_df_hum.as_matrix().reshape(-1, 1)
# weather_df_lat=weather_df_lat.as_matrix().reshape(-1, 1)

# Plotly regression tutorial : https://plot.ly/scikit-learn/plot-ols/

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(weather_df_lat, weather_df_hum)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(weather_df_lat) - weather_df_hum) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(weather_df_lat, weather_df_hum))

####
def data_to_plotly(x):
    k = []
    
    for i in range(0, len(x)):
        k.append(x[i][0])
        
    return k

####
p1 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=weather_df_hum, 
                mode='markers',
                text= weather_df.city + ", " + weather_df.country_code
               )

p2 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=regr.predict(weather_df_lat),
                mode='lines',
                text = "Regression Line",
                line=dict(color='red', width=3)
                )

layout = go.Layout(title='Humidity (%) over Latitude <br> (as of 08/25/2018) <br> Data Source: <a href="https://openweathermap.org/api">OpenWeatherMap</a>',
                xaxis=dict(title='Latitude',ticks='', 
               showticklabels=False,
                              zeroline=False),
                   yaxis=dict(title='Humidity (%)',ticks='', showticklabels=False,
                              zeroline=False),
                   showlegend=False, hovermode='closest')

fig = go.Figure(data=[p1, p2], layout=layout)

py.iplot(fig,session="knitr")

C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\site-packages\ipykernel_launcher.py:5: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



Coefficients: 
 [[0.08426053]]
Mean squared error: 512.53
Variance score: 0.01


In [12]:
# Cloudiness (%) vs. Latitude

# print(weather_df_clo.describe())

weather_df_clo=weather_df_clo.as_matrix().reshape(-1, 1)
# weather_df_lat=weather_df_lat.reshape(-1, 1)

# Plotly regression tutorial : https://plot.ly/scikit-learn/plot-ols/

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(weather_df_lat, weather_df_clo)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(weather_df_lat) - weather_df_clo) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(weather_df_lat, weather_df_clo))

####
def data_to_plotly(x):
    k = []
    
    for i in range(0, len(x)):
        k.append(x[i][0])
        
    return k

####
p1 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=weather_df_clo, 
                mode='markers',
                text= weather_df.city + ", " + weather_df.country_code
               )

p2 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=regr.predict(weather_df_lat),
                mode='lines',
                text = "Regression Line",
                line=dict(color='red', width=3)
                )

layout = go.Layout(title='Cloudiness (%) over Latitude <br> (as of 08/25/2018) <br> Data Source: <a href="https://openweathermap.org/api">OpenWeatherMap</a>',
                xaxis=dict(title='Latitude',ticks='', showticklabels=False,
                              zeroline=False),
                   yaxis=dict(title='Cloudiness (%)',ticks='', showticklabels=False,
                              zeroline=False),
                   showlegend=False, hovermode='closest')


fig = go.Figure(data=[p1, p2], layout=layout)

py.iplot(fig,session="knitr")

C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\site-packages\ipykernel_launcher.py:5: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



Coefficients: 
 [[0.10864511]]
Mean squared error: 1271.23
Variance score: 0.01


In [13]:
# Wind Speed (mph) vs. Latitude

# print(weather_df_wind.describe())

weather_df_wind=weather_df_wind.as_matrix().reshape(-1, 1)
# weather_df_lat=weather_df_lat.reshape(-1, 1)

# Plotly regression tutorial : https://plot.ly/scikit-learn/plot-ols/

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(weather_df_lat, weather_df_wind)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(weather_df_lat) - weather_df_wind) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(weather_df_lat, weather_df_wind))

####
def data_to_plotly(x):
    k = []
    
    for i in range(0, len(x)):
        k.append(x[i][0])
        
    return k

####
p1 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=weather_df_wind, 
                mode='markers',
                text= weather_df.city + ", " + weather_df.country_code
               )

p2 = go.Scatter(x=data_to_plotly(weather_df_lat), 
                y=regr.predict(weather_df_lat),
                mode='lines',
                line=dict(color='red', width=3),
                text = "Regression Line"

                )

layout = go.Layout(title='Wind Speeds (mph) over Latitude <br> (as of 08/25/2018) <br>Data Source: <a href="https://openweathermap.org/api">OpenWeatherMap</a>',
    xaxis=dict(title='Latitude',ticks='', showticklabels=False,
                              zeroline=False),
                   yaxis=dict(title='Wind Speeds (mph)',ticks='', showticklabels=False,
                              zeroline=False),
                   showlegend=False, hovermode='closest')

fig = go.Figure(data=[p1, p2], layout=layout)


py.iplot(fig,session="knitr")

C:\Users\White Base\AppData\Local\conda\conda\envs\FinalProject\lib\site-packages\ipykernel_launcher.py:5: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



Coefficients: 
 [[-0.00484205]]
Mean squared error: 4.93
Variance score: 0.00


In [9]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = go.Figure(
    data=[
        go.Scattergeo(
            lat=weather_df.Latitude,
            lon=weather_df.Longitude,
            marker={
                "color": 'blue',
                "line": {
                    "width": 1
                },
                "size": 5
            },
            mode="markers+text",
            name="",
#             text= weather_df.city + ', '+ weather_df.country_code,
            textposition=["top center"]
        )
    ],
    layout={
        "title": "Citipy's Random Sample of Cities",
        "geo": {
            "lataxis": {
                "range": [-90, 90]
            },
            "lonaxis": {
                "range": [-180, -180]
            },
            "scope": "world"
        }
    }
)



py.iplot(fig)

In [ ]:
# https://plot.ly/pandas/scatter-plots-on-maps/
# https://plot.ly/python/text-and-annotations/
# https://plot.ly/python/shapes/

In [54]:
weather_df.head()

,city,country_code,Latitude,Longitude,Temperature,Wind_speed,Humidity,Cloudiness
0,darhan,mn,49.49,105.92,5.23,1.16,86.0,0.0
1,kyra,ru,49.58,111.97,13.28,0.96,89.0,48.0
2,paraopeba,br,-19.28,-44.40,25.00,4.10,47.0,75.0
3,souillac,mu,-20.52,57.52,23.00,6.70,73.0,75.0
4,slave lake,ca,55.28,-114.77,15.43,1.81,72.0,76.0
